## Spam Filter Using USE (Universal Sentence Encoder)

Project Objective: To detect if a message is a Spam or Not (Ham)

Dataset Source: https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset

##### Import Necessary Libraries

In [0]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
from sklearn.metrics import classification_report, accuracy_score
import pyspark.sql.functions as F

##### Data Ingestion & Data Preprocessing

In [0]:
# File location and type
file_location = "/FileStore/tables/spam.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

df = df.withColumnRenamed("v1", "label")\
    .withColumnRenamed("v2", "text")

df = df.select(F.regexp_replace('label', '\"', '').alias('label'), 'text')

df = df.where("label in ('ham', 'spam')")

df = df.na.drop()

display(df)

label,text
ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..."
ham,Ok lar... Joking wif u oni...
spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
ham,U dun say so early hor... U c already then say...
ham,"Nah I don't think he goes to usf, he lives around here though"
spam,"FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, å£1.50 to rcv"
ham,Even my brother is not like to speak with me. They treat me like aids patent.
ham,As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune
spam,WINNER!! As a valued network customer you have been selected to receivea å£900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.
spam,Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030


Output can only be rendered in Databricks

##### Display Unique Label Values

In [0]:
unique_label_values = df.dropDuplicates(["label"]).select("label")
unique_label_values.show()

+-----+
|label|
+-----+
|  ham|
| spam|
+-----+



##### Split Dataset into Training & Testing Dataset

In [0]:
train_ds, test_ds = df.randomSplit(weights=[0.80,0.20], seed=42)

##### Instantiate Pipeline Stages

In [0]:
doc = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

use = UniversalSentenceEncoder.pretrained('tfhub_use', lang="en") \
    .setInputCols(["document"])\
    .setOutputCol("sent_embeds")

doc_clf = ClassifierDLModel.pretrained('classifierdl_use_spam', 'en') \
    .setInputCols(["document", "sent_embeds"]) \
    .setOutputCol("class")

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]
classifierdl_use_spam download started this may take some time.
Approximate size to download 21.3 MB
[OK!]


##### Build Pipeline

In [0]:
spam_nlp_pipeline = Pipeline().setStages([doc, use, doc_clf])

##### Fit/Train Model

In [0]:
spam_nlp_model = spam_nlp_pipeline.fit(train_ds)

##### Predict Values Based on Testing Dataset

In [0]:
predictions = spam_nlp_model.transform(test_ds)

##### Display Predictions

In [0]:
display(predictions)

label text document sent_embeds class ham <#> in mca. But not conform. List(List(document, 0, 35, <#> in mca. But not conform., Map(sentence -> 0), List())) List(List(sentence_embeddings, 0, 35, <#> in mca. But not conform., Map(sentence -> 0, token -> <#> in mca. But not conform., pieceId -> -1, isWordStart -> true), List(0.02746016, -0.02419157, -0.034793332, -0.004867619, -0.021875376, 0.008849617, -0.0069125807, -0.031138007, -0.019572973, -0.07207335, 0.062004216, -0.010563536, -0.029117892, 0.044262122, -0.090299554, 0.027263252, -0.029477343, 0.0421426, 0.052129705, -0.04507972, 0.008414994, -0.009524377, -0.05308538, -0.0119680455, -0.061011825, 0.06964988, -0.03608504, -0.022911575, -0.02652597, -0.052947775, -0.020785892, -0.009607924, -0.06356423, -0.011991104, -0.049959257, 0.07565309, 0.0795804, 0.026952967, -0.08499657, -0.01307772, -0.009879971, -0.042349022, -0.060487352, 0.0872066, -0.04717463, 0.0064317794, -0.041594107, -0.027852437, 0.02928132, -0.01322366, -0.06324962, -0.007169747, 0.06013172, 0.025475947, 0.021078864, 0.038641054, 0.010047623, -0.07116231, 0.0102834795, 0.04422229, 0.0055542034, -0.026280195, 0.002188685, -0.052058745, 0.06722679, 0.0038666078, 0.023496056, 0.04598397, -0.029317843, 0.07484785, 0.0016924923, -0.032187402, -0.07188707, 0.037063103, -0.020972999, -0.033467643, 0.035588656, 0.08903128, -0.067217, -0.07057875, 0.05596615, -0.012620228, 0.0036597366, -0.01760222, 0.0396867, 0.012859098, -0.0028061403, 0.044114903, -0.002828418, -0.05877225, 0.02845784, 0.019407699, -0.02953344, 0.023449376, -0.031391293, -0.025308797, -0.08581063, -0.0076019973, 0.043213245, -0.015765855, 0.0109802205, -0.014508211, 0.0036381918, 0.027025312, 0.0028850439, 0.07716988, 0.08796895, 0.0026189368, -0.03595726, 0.057026386, -0.006466485, -0.0508606, 0.029911475, -0.035041682, -0.0038342683, 0.0044393865, -0.055242535, -0.046314877, -0.018859118, 0.01051388, 0.0012468735, -0.029041609, 0.034331847, 0.038516223, 0.065159455, -0.03150366, 0.058040343, 0.027119406, 0.053775113, -0.004270321, 0.019910945, 0.057819087, 0.0032955445, 0.049359635, -0.0661615, -0.01165795, -0.010944072, -0.054700732, 0.04550208, 0.011156826, -0.080617934, -0.009963626, -0.058635116, 0.017218094, 0.0186813, 0.04264004, -0.041119885, -0.03791302, 0.036758766, 0.015268717, 0.009497773, -0.013624097, 0.04525228, -0.059888482, 0.037434794, -0.047177114, -0.068717696, -0.03609577, 0.067628935, -1.3585885E-4, -0.0416926, 0.027799726, 0.02795165, -0.029644288, 0.07681911, 0.032576643, 0.024559855, 0.08816011, -0.01167934, -0.022385292, 0.030517505, 0.037963692, -0.027398035, -0.014055456, 0.02745635, -0.043047287, -0.005682491, 0.037824932, -0.035866424, 0.059468113, -0.036432188, 0.010105322, -0.009196841, -0.011943611, -0.017635131, 0.049907535, 7.1421947E-4, -0.013873018, -0.04010925, -0.06040729, 0.009214876, -0.031420574, 0.0017179939, -0.05608463, -0.06402621, -0.06657049, 0.057394203, -0.037238143, 0.07660456, 0.038973574, 0.023582626, 0.0644272, -0.02815914, -0.04857958, 0.041093893, -0.017261172, -0.033400994, 0.0014098034, 0.041126307, 0.0176808, 0.035022423, 0.021683425, 0.03613195, 0.01377052, 0.028511604, -0.06337914, -0.08232145, 0.052539412, 0.00659021, 0.051120736, 0.09266693, 0.043574695, -0.023876166, -0.054496843, -0.035931878, 0.08928466, -0.061547432, -0.03971452, -0.056810882, 0.07415464, 0.056390576, -0.032426894, 0.0785953, 0.04220148, 0.016026517, 0.044616196, -0.012223809, -0.003526975, -0.008050981, -0.044786334, 0.08905077, -0.0137048, 0.029556878, 0.0015336118, -0.054735888, 0.050804283, -0.0020861197, 0.0017026308, -0.05893727, 0.05715724, 0.055646032, 0.0072039426, -0.07422485, -0.003998585, 0.048692808, 0.017511701, 0.042463314, -0.06024382, -0.05269401, -0.006429492, -0.0073315115, 0.05886828, 0.03622378, 0.014677184, 0.045263737, -0.0052092355, 0.02358822, -0.03155117, 0.054189797, -0.012692007, -0.029428994, 0.02192655, -0.0046678246, 0.037829217, -0.04381403, 0.050250184, -0.039321437, -0.050321

##### Convert Necessary Features to Pandas DataFrame

In [0]:
predictions_in_pandas = (predictions.select(F.col('text').alias("text"), F.col('label').alias("ground_truth"), F.col('class.result').alias("prediction"))).toPandas()
display(predictions_in_pandas)

text,ground_truth,prediction
<#> in mca. But not conform.,ham,List(ham)
"said kiss, kiss, i can't do the sound effects! He is a gorgeous man isn't he! Kind of person who needs a smile to brighten his day!",ham,List(ham)
"""7 wonders in My WORLD 7th You 6th Ur style 5th Ur smile 4th Ur Personality 3rd Ur Nature 2nd Ur SMS and 1st \Ur Lovely Friendship""""... good morning dear""""""",ham,List(ham)
"""And stop wondering \wow is she ever going to stop tm'ing me ?!"""" because I will tm you whenever I want because you are MINE ... *laughs*""""""",ham,List(ham)
"""Beautiful Truth against Gravity.. Read carefully: \Our heart feels light when someone is in it.. But it feels very heavy when someone leaves it.."""" GOODMORNING""""""",ham,List(ham)
"""Edison has rightly said, \A fool can ask more questions than a wise man can answer"""" Now you know why all of us are speechless during ViVa.. GM""",ham,List(ham)
"""GOD ASKED, \What is forgiveness?"""" A little child gave lovely reply""",ham,List(ham)
"""I'm not smoking while people use \wylie smokes too much"""" to justify ruining my shit""""""",ham,List(ham)
"""The word \Checkmate"""" in chess comes from the Persian phrase \""""Shah Maat\"""" which means; \""""the king is dead..\"""" Goodmorning.. Have a good day..:)""""""",ham,List(ham)
"""Today is \song dedicated day.."""" Which song will u dedicate for me? Send this to all ur valuable frnds but first rply me...""""""",ham,List(ham)


##### Instantiate & Display Classification Report

In [0]:
predictions_in_pandas['prediction'] = predictions_in_pandas['prediction'].apply(lambda x : x[0])

report = classification_report(predictions_in_pandas['ground_truth'], predictions_in_pandas['prediction'])
print(report)

              precision    recall  f1-score   support

         ham       0.99      0.99      0.99       923
        spam       0.96      0.92      0.94       148

    accuracy                           0.98      1071
   macro avg       0.97      0.96      0.96      1071
weighted avg       0.98      0.98      0.98      1071



##### Notes & Other Takeaways From This Project

****
- Running this project with Spark NLP resulted in approximately the same results (both had F1 scores in the 0.98-0.9849 range).
****
- This project took far less time to train than the project that utilized the HuggingFace Trainer API.
****
- Since this was one of my first projects using Apache Spark, it is not as sophisticated as I am use to. I have to start somewhere, but this was a great start!
****